In [0]:
!apt install -y ffmpeg


#Environment Setting

In [0]:

#List of dependencies
import os
import shutil
import copy
import pandas as pd
import datetime
import re
import time
import json

# from fuzzywuzzy import fuzz
# from fuzzywuzzy import process
import difflib
## mp4 to .wav converter
import ffmpeg

import os
import azure.cognitiveservices.speech as speechsdk

SPEECH_REGION = "xxxx"
SPEECH_KEY="xxxx"

#####################################################################################
############################## GLOBAL VARIABLE SETTING #############################
#####################################################################################
if not('IS_FIRST_NOTEBOOK_RUN' in locals()):
#     os.system("apt-get -f -y install xxxxx")
    IS_FIRST_NOTEBOOK_RUN = True

In [0]:

from pydub import AudioSegment
import argparse
import base64

#formats_to_convert = ['.m4a']
filename = dbutils.widgets.get('filename')


decoded_bytes = base64.b64decode(filename)
filename = decoded_bytes.decode('utf-8')


filename = filename.split("/")[1] #define the filename ###.aac. m4a ...
if filename.split(".")[0] not in ['csv','txt']:

    #if filename.endswith(tuple(formats_to_convert)):
    filepath = PATH+"audio_files"+"/" + filename
    (path, file_extension) = os.path.splitext(filepath)
    file_extension_final = file_extension.replace('.', '')
    try:
        if file_extension_final == 'opus':
            track = AudioSegment.from_file(filepath, format="ogg", codec="libopus")
        else:
            track = AudioSegment.from_file(filepath, file_extension_final)

        wav_filename = filename.replace(file_extension_final, 'wav')
        wav_path = PATH+'audio_wav_files/' + wav_filename
        file_handle = track.export(wav_path, format='wav')
        print('CONVERTING: ' + str(filepath))
        while not os.path.exists(wav_path):
            print(wav_path)
            file_handle = track.export(wav_path, format='wav')
    except:
        print("ERROR CONVERTING " + str(filepath))
else:
    pass

---------------------------------------------------------------------------
Py4JJavaError                             Traceback (most recent call last)
<command-2936792143812905> in <cell line: 5>()
      3 
      4 #formats_to_convert = ['.m4a']
----> 5 filename = dbutils.widgets.get('filename')
      6 filename = filename.split("/")[1] #define the filename ###.aac. m4a ...
      7 if filename.split(".")[0] not in ['csv','txt']:

/databricks/python_shell/dbruntime/WidgetHandlerImpl.py in get(self, name)
     40         :return: Current value of the widget or default value
     41         """
---> 42         return self._notebookArguments.getArgument(name, self._entry_point.getCurrentBindings())
     43 
     44     def getArgument(self, name, defaultValue=None):

/databricks/spark/python/lib/py4j-0.10.9.5-src.zip/py4j/java_gateway.py in __call__(self, *args)
   1319 
   1320         answer = self.gateway_client.send_command(command)
-> 1321         return_value = get_return_value(
   

#API Request

In [0]:
#https://learn.microsoft.com/en-us/azure/cognitive-services/speech-service/batch-transcription-get?pivots=rest-api

import requests
import json
import time



subscription_key="xxxx"
contenturl = '"xxxx"' + wav_filename
# contentsasurl = contenturl + '?' + sas_token
region =  "eastasia"
endpoint = f""xxxx""

def request_post(subscription_key,contenturl):

    headers = {
        "Ocp-Apim-Subscription-Key": subscription_key,
        "Content-Type": "application/json"
    }
    # Set the request payload
    payload = {
        "contentUrls": [
            contenturl,
        ],
        "locale": "ja-JP",
        "displayName": "My Transcription",
        "model": None,
        "properties": {
            "diarizationEnabled":True,
            "wordLevelTimestampsEnabled": True,
            "punctuationMode": "DictatedAndAutomatic",
            "profanityFilterMode": "Masked",
            "diarization": {
                "speakers": {
                    "minCount": 2,
                    "maxCount": 3}
                    }
        }
    }

    # Send the POST request
    response = requests.post(endpoint, headers=headers, json=payload)

    # Check the response status
    if response.status_code == 201:
        print("Transcription request submitted successfully.")
        # Extract the transcription ID from the response headers
        location_header = response.headers.get("location")
        if location_header:
            transcription_id = location_header.split("/")[-1]
            return transcription_id
        else:
            return "Error submitting transcription request."
    else:
        return "Error submitting transcription request."



In [0]:
def get_response(transcription_id): #successed or not
    endpoint = f""xxxx"/{transcription_id}"

    # Set the request headers
    headers = {
        "Ocp-Apim-Subscription-Key": subscription_key
    }

    # Send the GET request
    response = requests.get(endpoint, headers=headers)

    # Check the response status
    if response.status_code == 200:
        # Get the result files from the response JSON
        status = response.json()['status'] 
        return status
    else:
        return "Error retrieving result files_Step_1."
    

In [0]:
def get_files(transcription_id): 
    endpoint = f""xxxx"/{transcription_id}/files"

    # Set the request headers
    headers = {"Ocp-Apim-Subscription-Key": subscription_key}
    # Send the GET request
    response = requests.get(endpoint, headers=headers)
    # Check the response status
    if response.status_code == 200:
        # Get the result files from the response JSON
        result_files = response.json()
        return result_files
    else:
        return "Error retrieving result files."


In [0]:

def get_link_content(results):
    # JSON link
    json_url = results['values'][0]['links']['contentUrl']

    # Make an HTTP GET request
    link_response = requests.get(json_url)

    # Check if the request was successful (status code 200)
    if link_response.status_code == 200:
        # Retrieve the JSON data
        json_data = link_response.json()
        return json_data

    else:
        return "Failed to retrieve JSON data. Status code:" + link_response.status_code


In [0]:
def content(content_json):
    try:
        content_json = get_link_content(results)
        content_json = content_json['recognizedPhrases']

        output = []
        previous_speaker = None
        if len(content_json) > 0:
            for i in range(len(content_json)):
                speaker = content_json[i]['speaker']
                text = content_json[i]['nBest'][0]['display']

                if speaker == previous_speaker:
                    output[-1] += " " + "Speaker " + str(speaker) + ": " + text
                else:
                    output.append("Speaker " + str(speaker) + ": " + text)
                
                previous_speaker = speaker

        content_output = "\n".join(output)
        return content_output

    except:
        return 'Error to get link_content'

In [0]:

request_post = request_post(subscription_key,contenturl)
if request_post != "Error submitting transcription request.":
    transcription_id = request_post

    if get_response(transcription_id) != "Error retrieving result files_Step_1.":
        times = 0
        while times < 20:
            if get_response(transcription_id) == 'Succeeded':
                results = get_files(transcription_id)
                break
            else:
                time.sleep(30)
                continue
                times += 1
        if results != "Error retrieving result files.":
            content_json = get_link_content(results)
            if content(content_json) != 'Error to get link_content':
                content = content(content_json)
            else:
                print('Error to get link_content')
        else:
            print("Error retrieving result files.")
    else:
        print("Error retrieving result files_Step_1.")
else:
    print("Error for request submit.")

Transcription request submitted successfully.


In [0]:
with open(PATH+'transcript_for_each_audio/'+filename[:-4]+".txt", 'w') as f: #overwrite the file if it existing
    if content == 0:
        content= ""
    f.write(content)



#OpenAI Summary

In [0]:
import base64
import requests
import json

url="xxxx"

payload = json.dumps({
"user": "[User GAD]",
"model":"gpt-35-turbo",
"messages": [
{
"role": "system",
"content": "You are a helpful assistant."
},
{
"role": "user",
"content": f""""xxxx" """
}
]
})

client_key="xxxx"
client_secret="xxxx"

id_secret_bytes = client_key + ':' + client_secret
encoded_u = base64.b64encode(id_secret_bytes.encode()).decode()
headers = {"Authorization": "Basic %s" % encoded_u, 'Content-type': 'application/json', 'Accept': 'application/json'}

r=requests.post(url,headers=headers,data=payload)
print("RESPONSE"+r.text)

In [0]:
content_summary = r.json()['choices'][0]['message']['content']

In [0]:

content_with_summary = content + "\n" + "Conservation_Summary:" + content_summary
with open(PATH +'transcript_for_each_audio/' +filename[:-4]+".txt", 'w') as f: #overwrite the file if it existing
    if content == 0:
        content= ""
    f.write(content_with_summary)





In [0]:
from datetime import datetime
from datetime import timedelta, timezone

df_file_time = pd.DataFrame(columns = ['FileName','FileTime'])
for (dirpath, dirnames, filenames) in os.walk(PATH+"audio_files"+"/"):
    for filename in filenames:
        if filename.split('.')[1] not in ['csv','txt']:
            filepath = dirpath + filename
            filetime = os.path.getctime(filepath)
            
filename_list = []
filetime_list = []
TAT_list = []
now = datetime.now()
for (dirpath, dirnames, filenames) in os.walk(PATH+"audio_files"+"/"):
    for filename in filenames:
        if filename.split('.')[1] not in ['csv','txt']:
            filename_list.append(filename.split(".")[0])
            filepath = dirpath + filename
            filetime = os.path.getctime(filepath)
            filetime = datetime.fromtimestamp(filetime)
            # Create the UTC+7 timezone
            utc_plus_7 = timezone(timedelta(hours=7))
            filetime = filetime.astimezone(utc_plus_7)
            time_gap = now.timetuple().tm_yday - filetime.timetuple().tm_yday
            filetime = filetime.strftime("%Y-%m-%d %H:%M:%S")
            filetime_list.append(filetime)
            TAT_list.append(time_gap)
            
df_file_time =  pd.DataFrame({'FileName':filename_list,'FileTime':filetime_list})
df_file_time 

In [0]:
#text_folders = os.listdir(PATH)
text_folders = os.listdir(PATH +'transcript_for_each_audio/')
speech_list = []
speech_name = []
speech_summary = []

for file in text_folders:
    split_tup = os.path.splitext(file)
    if split_tup[0] != 'temp':
        if split_tup[1] == ".txt":
            with open(PATH +'transcript_for_each_audio/'+file) as f:
                speech = f.read()
            speech_list.append(speech.split('Conservation_Summary:')[0])
            try:
                speech_summary.append(speech.split('Conservation_Summary:')[1])
            except:
                speech_summary.append('Error')
            speech_name.append(split_tup[0])


df = pd.DataFrame({'FileName':speech_name,
                  'Text':speech_list,
                  'Summary':speech_summary})       


df['Manual Review'] = ''


#Data to DB

In [0]:
%sh
curl https://packages.microsoft.com/keys/microsoft.asc | apt-key add -
curl https://packages.microsoft.com/config/ubuntu/16.04/prod.list > /etc/apt/sources.list.d/mssql-release.list 
apt-get update
ACCEPT_EULA=Y apt-get install msodbcsql17
apt-get -y install unixodbc-dev
sudo apt-get install python3-pip -y
pip3 install --upgrade pyodbc

In [0]:
dbutils.fs.put("/databricks/init/<YourClusterName>/pyodbc-install.sh","""
#!/bin/bash
sudo apt-get update
sudo apt-get -q -y install unixodbc unixodbc-dev
sudo apt-get -q -y install python3-dev
/databricks/python/bin/pip install pyodbc
""", True)

import urllib
import pyodbc
for driver in pyodbc.drivers():
    print(driver)

from sqlalchemy import create_engine

server = "xxxx"
database = 
username = 
password = 

driver = '{ODBC Driver 17 for SQL Server}'

odbc_str = 'DRIVER='+driver+';SERVER='+server+';PORT=1433;UID='+username+';DATABASE='+ database + ';PWD='+ password
connect_str = 'mssql+pyodbc:///?odbc_connect=' + urllib.parse.quote_plus(odbc_str)
engine = create_engine(connect_str)

#used to create a table, will overwrite 
def to_sql(df, table):
    df.to_sql(table, engine, if_exists = "replace", index=False, chunksize = 100)


In [0]:
conn = pyodbc.connect(odbc_str)
sql = 'SELECT * FROM [dbo].["xxxx"]'
df = pd.read_sql(sql, conn)

In [0]:
import pandas as pd
import datetime
now = datetime.datetime.now()
now = now.strftime("%Y-%m-%d")
df_result.to_csv(PATH+"transcript_version"xxxx"_"+now+".csv",index=False,encoding='utf-16', sep='\t')


In [0]:
table_name = '"xxxx"'
cursor = conn.cursor()
if cursor.tables(table=table_name, tableType='TABLE').fetchone():
    delete_table_query = f"DROP TABLE {table_name}"
    cursor.execute(delete_table_query)
    conn.commit()

# Create a new table
create_table_query = f"""
CREATE TABLE {table_name} (
    FileName NVARCHAR(MAX),
    Text NVARCHAR(MAX),
    Summary NVARCHAR(MAX),
    FileTime VARCHAR(255),
    [Manual Review] VARCHAR(255)
)
"""

cursor = conn.cursor()
cursor.execute(create_table_query)
conn.commit()

##Insert data
insert_query = f"""
INSERT INTO {table_name} (FileName, Text, Summary, FileTime, [Manual Review])
VALUES (?, ?, ?, ?, ?)
"""
data_tuples = [tuple(row) for row in df_result.to_numpy()]
cursor.executemany(insert_query, data_tuples)
conn.commit()



In [0]:
try:
    # create primary key
    conn = pyodbc.connect(odbc_str)
    cur = conn.cursor()
    sql = '''ALTER TABLE "xxxx" ADD id INT NOT NULL PRIMARY KEY IDENTITY'''
    cur.execute(sql)
    conn.commit()
except:
    # create primary key
    conn = pyodbc.connect(odbc_str)
    cur = conn.cursor()
    sql = '''ALTER TABLE "xxxx" ADD id INT NOT NULL PRIMARY KEY IDENTITY'''
    cur.execute(sql)
    conn.commit()
    